In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
#pip install netaddr

In [4]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [5]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Tuesday_Packet_Level_Data.pkl')

In [6]:
df1 = wed_data.copy()

In [7]:
del wed_data

In [8]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [9]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499170800.000003000,13.107.4.50,192.168.10.14,80,50128,5612,211623539,b8ac6f3607ee00c1b114eb310800,450005dc468640007706db420d6b0432c0a80a0e,0050c3d06674f9b8d6f187ea501003ffce740000,2d04ab43650cdc5e7f37f485d64ab09fa83bb3ab0389c7...,bwd,1.0,6.0
1,1499170800.000123000,192.168.10.14,13.107.4.50,50128,80,5612,20954,00c1b114eb31b8ac6f3607ee0800,4500002874b540008006a9c7c0a80a0e0d6b0432,c3d00050d6f187ea66750520501006273ec80000,000000000000,bwd,1.0,6.0
2,1499170800.000196000,13.107.4.50,192.168.10.14,80,50128,5612,211624999,b8ac6f3607ee00c1b114eb310800,45000b90468740007706d58d0d6b0432c0a80a0e,0050c3d06674ff6cd6f187ea501003ffe7d50000,189fa2c3610086100fd908edc783be4c1cbc80e041b84c...,bwd,1.0,6.0
3,1499170800.000388000,13.107.4.50,192.168.10.14,80,50128,5612,211627919,b8ac6f3607ee00c1b114eb310800,45000b90468940007706d58b0d6b0432c0a80a0e,0050c3d066750ad4d6f187ea501003ffe7d50000,cab71d48416f5ed737efe031bae17133f7c660084eecbe...,fwd,1.0,6.0
4,1499170800.000430000,192.168.10.14,13.107.4.50,50128,80,5612,20954,00c1b114eb31b8ac6f3607ee0800,4500002874b640008006a9c6c0a80a0e0d6b0432,c3d00050d6f187ea6675163c501006272dac0000,000000000000,bwd,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072428,1499171464.556106000,192.168.10.50,172.16.0.1,21,53654,10076,55,00c1b114eb310019b90a69f10800,4500003414d740004006af01c0a80a32ac100001,0015d19620052f8459de7112801000e38e2f0000010108...,00,bwd,0.0,0.0
4072429,1499171464.556259000,13.107.4.50,192.168.10.15,80,50580,9011,657167079,001e4fd4ca2800c1b114eb310800,45001144038c4000760613d40d6b0432c0a80a0f,0050c5943afc9ef6c6ab5e93501003feed8a0000,4c3b16f75ac96e7a8feca0e6cf2fa31bbf97915f49cbe4...,bwd,0.0,0.0
4072430,1499171464.556366000,13.107.4.50,192.168.10.15,80,50580,9011,657171459,001e4fd4ca2800c1b114eb310800,450005dc038f400076061f390d6b0432c0a80a0f,0050c5943afcb012c6ab5e93501003fe95ed0000,29d25be7dd2726626b7eff41d89a7db674f277ad59db65...,bwd,0.0,0.0
4072431,1499171464.556645000,13.107.4.50,192.168.10.15,80,50580,9011,657172919,001e4fd4ca2800c1b114eb310800,450005dc0390400076061f380d6b0432c0a80a0f,0050c5943afcb5c6c6ab5e93501003fee5770000,2b2b51cf9e9dc9c77d74bbbdd24ffa634b22ee46f2e8bf...,bwd,0.0,0.0


In [10]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [11]:
df1['Class Label'].value_counts()

0.0    3798386
7.0     163311
6.0     110736
Name: Class Label, dtype: int64

In [12]:
df = df1[df1['Class Label']== 6]

In [13]:
df = df.reset_index(inplace = False)

In [14]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,0,1499170800.000003000,13.107.4.50,192.168.10.14,80,50128,5612,211623539,b8ac6f3607ee00c1b114eb310800,450005dc468640007706db420d6b0432c0a80a0e,0050c3d06674f9b8d6f187ea501003ffce740000,2d04ab43650cdc5e7f37f485d64ab09fa83bb3ab0389c7...,bwd,1.0,6.0
1,1,1499170800.000123000,192.168.10.14,13.107.4.50,50128,80,5612,20954,00c1b114eb31b8ac6f3607ee0800,4500002874b540008006a9c7c0a80a0e0d6b0432,c3d00050d6f187ea66750520501006273ec80000,000000000000,bwd,1.0,6.0
2,2,1499170800.000196000,13.107.4.50,192.168.10.14,80,50128,5612,211624999,b8ac6f3607ee00c1b114eb310800,45000b90468740007706d58d0d6b0432c0a80a0e,0050c3d06674ff6cd6f187ea501003ffe7d50000,189fa2c3610086100fd908edc783be4c1cbc80e041b84c...,bwd,1.0,6.0
3,3,1499170800.000388000,13.107.4.50,192.168.10.14,80,50128,5612,211627919,b8ac6f3607ee00c1b114eb310800,45000b90468940007706d58b0d6b0432c0a80a0e,0050c3d066750ad4d6f187ea501003ffe7d50000,cab71d48416f5ed737efe031bae17133f7c660084eecbe...,fwd,1.0,6.0
4,4,1499170800.000430000,192.168.10.14,13.107.4.50,50128,80,5612,20954,00c1b114eb31b8ac6f3607ee0800,4500002874b640008006a9c6c0a80a0e0d6b0432,c3d00050d6f187ea6675163c501006272dac0000,000000000000,bwd,1.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110731,110731,1499170823.617612000,192.168.10.15,23.194.141.146,49615,80,6290,2047,00c1b114eb31001e4fd4ca280800,450000287a3f400080061085c0a80a0f17c28d92,c1cf00505a53c03b10ed8b3d501003c9c3260000,000000000000,fwd,1.0,6.0
110732,110732,1499170823.617783000,23.194.141.146,192.168.10.15,80,49615,6290,478108,001e4fd4ca2800c1b114eb310800,450805dc8b1c4000370642ec17c28d92c0a80a0f,0050c1cf10ed8b3d5a53c03b5010045a99740000,56d240122e49c80317357c5203cc09844a5420099a8490...,bwd,1.0,6.0
110733,110733,1499170823.617918000,192.168.10.15,23.194.141.146,49615,80,6290,2047,00c1b114eb31001e4fd4ca280800,450000287a40400080061084c0a80a0f17c28d92,c1cf00505a53c03b10ed96a5501003c9b7be0000,000000000000,fwd,1.0,6.0
110734,110734,1499170823.617974000,23.194.141.146,192.168.10.15,80,49615,6290,479568,001e4fd4ca2800c1b114eb310800,450805dc8b1d4000370642eb17c28d92c0a80a0f,0050c1cf10ed90f15a53c03b5010045a19210000,1792ccc5be80c30aa0636ce379cc2fdbf6723b8db9cdda...,bwd,1.0,6.0


In [15]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [16]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [17]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [18]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [19]:
# 70% of benign data for train, 30% for test benign

In [20]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [21]:
#len(imlist)

In [22]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [23]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [24]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [25]:
#len(streams_lst1)

In [26]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,13.107.4.50,192.168.10.14,80,50128,1499170800.000003000,5612,211623539,bwd,"[0.0196078431372549, 0.8627450980392157, 0.274..."
1,192.168.10.14,13.107.4.50,50128,80,1499170800.000123000,5612,20954,bwd,"[0.0, 0.1568627450980392, 0.4549019607843137, ..."
2,13.107.4.50,192.168.10.14,80,50128,1499170800.000196000,5612,211624999,bwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."
3,13.107.4.50,192.168.10.14,80,50128,1499170800.000388000,5612,211627919,fwd,"[0.043137254901960784, 0.5647058823529412, 0.2..."
4,192.168.10.14,13.107.4.50,50128,80,1499170800.000430000,5612,20954,bwd,"[0.0, 0.1568627450980392, 0.4549019607843137, ..."
...,...,...,...,...,...,...,...,...,...
110731,192.168.10.15,23.194.141.146,49615,80,1499170823.617612000,6290,2047,fwd,"[0.0, 0.1568627450980392, 0.47843137254901963,..."
110732,23.194.141.146,192.168.10.15,80,49615,1499170823.617783000,6290,478108,bwd,"[0.0196078431372549, 0.8627450980392157, 0.545..."
110733,192.168.10.15,23.194.141.146,49615,80,1499170823.617918000,6290,2047,fwd,"[0.0, 0.1568627450980392, 0.47843137254901963,..."
110734,23.194.141.146,192.168.10.15,80,49615,1499170823.617974000,6290,479568,bwd,"[0.0196078431372549, 0.8627450980392157, 0.545..."


In [27]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Tuesday_ftp_all/'

for PKT in range(2, 16):
    
    file_name = f'tuesday_ftp_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 475
